In [1]:
import csv 
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time

base_url = "https://qazaqrepublic.com"
search_url = "https://qazaqrepublic.com/shop"

apartment_links = []

for page_num in range(0, 100):
    current_page_url = f"{search_url}?p={page_num}"

    response = requests.get(current_page_url)
    time.sleep(1)
    soup = BeautifulSoup(response.text, 'html.parser')

    listings = soup.find_all('a', {'class': 'catalog_item'})
    if not listings:
        break  # No more listings, exit the loop

    for item in listings:
        link = item.get('href')  # Directly get 'href' attribute from 'item'
        apartment_links.append(base_url + link)


# Print the total number of links found
print(f"Found {len(apartment_links)} apartment links.")

with open('qr.csv', 'a', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    for apartment_link in apartment_links:
        csv_writer.writerow([apartment_link])


Found 21100 apartment links.


In [4]:
df = pd.read_csv('qr.csv')
df.shape

(21099, 1)

In [5]:
df = df.drop_duplicates()
df.shape

(207, 1)

In [6]:
df.to_csv('qr_notdubl.csv', index=False)

In [76]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def qr_data(url):
    response = requests.get(url)
    time.sleep(1)
    soup = BeautifulSoup(response.text, 'html.parser')
    about_dict = {}
    qrs = soup.findAll('div', attrs={'class': 'page'})
    qr_tip = soup.find_all('div', class_='item_content')
    
    for qr in qr_tip:
        qr_name = qr.find("div", attrs={'class': 'item__name'}).text.split(' ', 1)[0]
        qr_type = qr.find("div", attrs={'class': 'item__name'}).text.strip(' ')
        qr_color = qr.find("div", attrs={'class': 'item__name'}).text.split(' ')[-1]
        qr_price = qr.find("div", attrs={'class': 'item__price'}).text.strip('')
        
    total_dict = {"Name": qr_name,
                  "Type": qr_type,
                  "Color": qr_color,
                  "Price": qr_price}
    
    total_dict.update(about_dict)
    return total_dict



In [80]:
data = []
df = pd.read_csv('qr_notdubl.csv')

links = df.iloc[:, 0].values
i = 1
for link in links[0:10]:
    try:
        print(i, link)
        i += 1
        qr_result = qr_data(link)
        data.append(qr_result)
    except Exception as e:
        print(f"Failed to extract data from {link}. Error: {e}")

1 https://qazaqrepublic.com/shop/sweatshirts/1007
2 https://qazaqrepublic.com/shop/hoodies/1124
3 https://qazaqrepublic.com/shop/bags/844
4 https://qazaqrepublic.com/shop/t-shirts/1038
5 https://qazaqrepublic.com/shop/t-shirts/1028
6 https://qazaqrepublic.com/shop/t-shirts/1144
7 https://qazaqrepublic.com/shop/t-shirts/1048
8 https://qazaqrepublic.com/shop/t-shirts/1146
9 https://qazaqrepublic.com/shop/t-shirts/1112
10 https://qazaqrepublic.com/shop/t-shirts/1106


In [81]:
df = pd.DataFrame(data, columns=['Name', 'Type','Color', 'Price'])
df.to_csv("qrs_data.csv", index=False) 

In [82]:
df = pd.read_csv("qrs_data.csv")
df.head(5)

,Name,Type,Color,Price
0,КЕҢЖЕЙДЕ,КЕҢЖЕЙДЕ jaqsy künder СҰР МЕЛАНЖ,МЕЛАНЖ,18000₸
1,БҮРЖЕЙДЕ,БҮРЖЕЙДЕ Born qazaq ҚОҢЫР,ҚОҢЫР,22000₸
2,ДОРБА,ДОРБА B1 ҚАРА,ҚАРА,21000₸
3,ФУТБОЛКА,ФУТБОЛКА QRD АҚ,АҚ,9000₸
4,ФУТБОЛКА,ФУТБОЛКА QRD КҮЛ ТҮСТЕС,ТҮСТЕС,9000₸
